In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from scipy.fftpack import fftn, fftshift
from skimage.filters import sobel
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray
from skimage import feature, filters, measure

# Función para cargar una imagen en escala de grises
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("No se pudo cargar la imagen")
    return img

# Función para combinar múltiples imágenes 2D en una imagen 3D
def stack_images_to_3d(folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.tif'))])
    if not image_files:
        raise ValueError("No se encontraron imágenes en la carpeta")
    
    images = [load_image(os.path.join(folder_path, f)) for f in image_files]
    volume = np.stack(images, axis=-1)  # Apila las imágenes en la tercera dimensión
    return volume

# Función para calcular la Transformada de Fourier 3D de la imagen
def compute_fft_3d(img):
    f_transform = fftn(img)  # Calcula la transformada de Fourier en 3D
    f_transform_shifted = fftshift(f_transform)  # Centra la transformada de Fourier
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  # Escala logarítmica para visualización
    return magnitude_spectrum

# Función para calcular el tensor de estructura, que mide la variación direccional de la imagen
def compute_structure_tensor(img):
    gradient_x = sobel(img, axis=0)  # Derivada en la dirección X
    gradient_y = sobel(img, axis=1)  # Derivada en la dirección Y
    gradient_z = sobel(img, axis=2)  # Derivada en la dirección Z
    
    # Cálculo de los elementos del tensor de estructura
    Jxx = gradient_x * gradient_x
    Jyy = gradient_y * gradient_y
    Jzz = gradient_z * gradient_z
    Jxy = gradient_x * gradient_y
    Jxz = gradient_x * gradient_z
    Jyz = gradient_y * gradient_z
    
    # Determinante de la matriz de orientación
    DoI = (Jxx + Jyy + Jzz) - np.sqrt((Jxx - Jyy)**2 + (Jyy - Jzz)**2 + (Jzz - Jxx)**2 + 4*(Jxy**2 + Jxz**2 + Jyz**2))
    return DoI

# Función para detectar bordes en cada capa 2D de una imagen 3D usando el algoritmo de Canny
def detect_edges_3d(img_3d):
    edges_3d = np.zeros_like(img_3d)
    for i in range(img_3d.shape[2]):  # Iterar sobre las capas Z
        edges_3d[:, :, i] = feature.canny(img_3d[:, :, i], sigma=1)
    return edges_3d

# Función para analizar la textura en cada capa 2D y calcular la media
def texture_analysis_3d(img_3d):
    contrast_values = []
    energy_values = []
    for i in range(img_3d.shape[2]):  # Iterar sobre las capas Z
        img_gray = (img_3d[:, :, i] / img_3d.max() * 255).astype(np.uint8)
        glcm = graycomatrix(img_gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
        contrast_values.append(graycoprops(glcm, 'contrast').mean())
        energy_values.append(graycoprops(glcm, 'energy').mean())
    return np.mean(contrast_values), np.mean(energy_values)

# Función principal para ejecutar el análisis de una carpeta de imágenes
def main(folder_path):
    img_3d = stack_images_to_3d(folder_path)  # Crear imagen 3D desde imágenes en la carpeta
    fft_result = compute_fft_3d(img_3d)  # Calcular la transformada de Fourier 3D
    structure_tensor = compute_structure_tensor(img_3d)  # Calcular el tensor de estructura
    edges = detect_edges_3d(img_3d)  # Detectar bordes con Canny en 3D
    contrast, energy = texture_analysis_3d(img_3d)  # Análisis de textura
    
    # Imprimir los resultados del análisis de textura
    print(f"Contraste de textura: {contrast:.3f}, Energía de textura: {energy:.3f}")
    
    # Visualización de los resultados
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 4, 1)
    plt.imshow(img_3d[:, :, img_3d.shape[2] // 2], cmap='gray')
    plt.title('Imagen 3D (Corte Medio)')
    plt.axis('off')
    
    plt.subplot(1, 4, 2)
    plt.imshow(fft_result[:, :, fft_result.shape[2] // 2], cmap='inferno')
    plt.title('Transformada de Fourier 3D')
    plt.axis('off')
    
    plt.subplot(1, 4, 3)
    plt.imshow(structure_tensor[:, :, structure_tensor.shape[2] // 2], cmap='viridis')
    plt.title('Tensor de Estructura')
    plt.axis('off')
    
    plt.subplot(1, 4, 4)
    plt.imshow(edges[:, :, edges.shape[2] // 2], cmap='gray')
    plt.title('Detección de Bordes 3D')
    plt.axis('off')
    
    plt.show()



In [ ]:
main("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")

In [ ]:
# Función para calcular la Transformada de Fourier 3D de la imagen
def compute_fft_3d(img):
    f_transform = fftn(img)  # Calcula la transformada de Fourier en 3D
    f_transform_shifted = fftshift(f_transform)  # Centra la transformada de Fourier
    magnitude_spectrum = np.log1p(np.abs(f_transform_shifted))  # Escala logarítmica para visualización
    return f_transform_shifted, magnitude_spectrum

# Función para detectar las direcciones principales en el espectro de Fourier
def detect_main_directions(magnitude_spectrum):
    # Encuentra los índices de los picos en el espectro de magnitud
    indices = np.unravel_index(np.argsort(magnitude_spectrum.ravel())[::-1], magnitude_spectrum.shape)
    main_directions = list(zip(indices[0][:2], indices[1][:2], indices[2][:2]))
    return main_directions

def filter_main_directions(f_transform, main_directions):
    filtered_transform = np.zeros_like(f_transform)
    for direction in main_directions:
        filtered_transform[direction] = f_transform[direction]
    return filtered_transform


def filter_main_directions_with_mask(f_transform, main_directions, radius=5):
    # Crear una máscara vacía del mismo tamaño que la transformada
    mask = np.zeros_like(f_transform, dtype=bool)
    
    # Rellenar la máscara alrededor de las direcciones principales
    for direction in main_directions:
        z, y, x = direction
        # Crear una esfera o región alrededor de la dirección principal
        z_min, z_max = max(0, z - radius), min(f_transform.shape[0], z + radius + 1)
        y_min, y_max = max(0, y - radius), min(f_transform.shape[1], y + radius + 1)
        x_min, x_max = max(0, x - radius), min(f_transform.shape[2], x + radius + 1)
        
        # Aplica la máscara en el rango definido
        mask[z_min:z_max, y_min:y_max, x_min:x_max] = True
    
    # Aplicar la máscara al espectro de Fourier
    filtered_transform = np.zeros_like(f_transform)
    filtered_transform[mask] = f_transform[mask]
    
    return filtered_transform

def filter_multiple_directions(f_transform, direction_vectors, tolerance=5):
    """
    Filtro para capturar múltiples direcciones principales en el espectro de Fourier en 3D.

    Parámetros:
    - f_transform: Transformada de Fourier en 3D.
    - direction_vectors: Lista de vectores normalizados que indican las direcciones deseadas.
    - tolerance: Rango angular (en grados) alrededor de las direcciones especificadas.
    """
    # Crear una máscara vacía del mismo tamaño que la transformada
    mask = np.zeros_like(f_transform, dtype=bool)

    # Iterar sobre cada dirección y combinar las máscaras
    for direction_vector in direction_vectors:
        # Normalizar el vector de dirección
        direction_vector = direction_vector / np.linalg.norm(direction_vector)
        
        # Crear las coordenadas del espacio
        z, y, x = np.indices(f_transform.shape)
        center = np.array(f_transform.shape) // 2
        z, y, x = z - center[0], y - center[1], x - center[2]  # Centrar las coordenadas
        
        # Calcular los vectores unitarios
        vectors = np.stack((x, y, z), axis=-1)
        norms = np.linalg.norm(vectors, axis=-1, keepdims=True) + 1e-8  # Evitar divisiones por 0
        unit_vectors = vectors / norms
        
        # Producto escalar entre los vectores unitarios y la dirección deseada
        dot_products = np.dot(unit_vectors.reshape(-1, 3), direction_vector)
        dot_products = dot_products.reshape(f_transform.shape)
        
        # Convertir la tolerancia a coseno del ángulo
        tolerance_cosine = np.cos(np.radians(tolerance))
        
        # Crear la máscara para esta dirección
        mask_direction = dot_products >= tolerance_cosine
        
        # Combinar la máscara actual con la máscara total
        mask |= mask_direction

    # Aplicar la máscara al espectro
    filtered_transform = np.zeros_like(f_transform)
    filtered_transform[mask] = f_transform[mask]
    
    return filtered_transform



#Función principal para ejecutar el análisis de una carpeta de imágenes
def filtrado(folder_path):
    img_3d = stack_images_to_3d(folder_path)  # Crear imagen 3D desde imágenes en la carpeta
    f_transform_shifted, magnitude_spectrum = compute_fft_3d(img_3d)  # Calcular la transformada de Fourier 3D
    
    main_directions = detect_main_directions(magnitude_spectrum)  # Detectar las direcciones principales
    filtered_transform = filter_multiple_directions(f_transform_shifted, main_directions)  # Filtrar las direcciones principales
    
    # Calcular el espectro de magnitud del espectro filtrado
    filtered_magnitude_spectrum = np.log1p(np.abs(filtered_transform))
    for i in range(img_3d.shape[2]):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(img_3d[:, :, i], cmap='gray')
        plt.title(f'Imagen 3D (Corte {i})')
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(magnitude_spectrum[:, :, i], cmap='turbo')
        plt.title(f'Transformada de Fourier 3D (Corte {i})')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(filtered_magnitude_spectrum[:, :, i], cmap='inferno')
        plt.title(f'Espectro Filtrado (Corte {i})')
        plt.axis('off')
        
        plt.show()
filtrado("C:\\Users\\danwo\\Desktop\\ZS0001_TI_25XW_Rhodopsin_Au")